In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'
dataset = pd.read_csv(r"./Datasets/HSI/HSI_MFADCCA.csv")
df = dataset.reset_index(drop=True)
df

,Unnamed: 0.1,Unnamed: 0,Date,Open,High,Low,Close,QPLp1,QPLp2,QPLp3,...,QPLn19,QPLn20,QPLn21,vol,log_returns,volatility_increments,log_volumes,hurst,hurstp,hurstm
0,0,262.0,2011/1/17,24364.07,24364.07,24098.03,24156.97,24422.49402,24559.83639,24730.45893,...,20388.49679,20154.18769,19921.45033,1.21,-0.005213,-0.029823,-0.234647,0.362061,0.490635,0.132432
1,1,263.0,2011/1/18,24149.44,24382.01,24104.34,24153.98,24207.34935,24343.48183,24512.60130,...,20208.88874,19976.64374,19745.95663,1.29,-0.000124,-0.121041,0.064022,0.452442,0.452977,0.555551
2,2,264.0,2011/1/19,24321.25,24434.40,24243.88,24419.62,24379.57134,24516.67233,24686.99499,...,20352.66388,20118.76658,19886.43826,1.36,0.010938,-0.483003,0.052842,0.463389,0.731069,0.332243
3,3,265.0,2011/1/20,24248.32,24248.32,23963.86,24003.70,24306.46646,24443.15633,24612.96827,...,20291.63413,20058.43820,19826.80654,1.53,-0.017179,0.410053,0.117783,0.472777,0.144175,0.658491
4,4,266.0,2011/1/21,23913.19,24130.16,23840.29,23876.86,23970.53283,24105.33355,24272.79855,...,20011.18851,19781.21552,19552.78518,1.34,-0.005298,0.167005,-0.132598,0.465770,0.077713,0.612136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1706,1706,1968.0,2017/12/19,29193.74,29339.11,29144.17,29253.66,29263.74537,29428.31300,29632.75791,...,24430.09211,24149.33611,23870.46341,1.68,0.006972,-0.033817,-0.005935,0.507095,0.442935,0.644336
1707,1707,1969.0,2017/12/20,29199.29,29326.52,29159.28,29234.09,29269.30867,29433.90760,29638.39137,...,24434.73650,24153.92712,23875.00140,1.35,-0.000669,-0.190234,-0.218689,0.597833,0.162749,0.851682
1708,1708,1970.0,2017/12/21,29209.64,29431.81,29145.13,29367.06,29279.68349,29444.34076,29648.89701,...,24443.39765,24162.48874,23883.46415,1.49,0.004538,-0.075585,0.098672,0.615340,0.727702,0.396648
1709,1709,1971.0,2017/12/22,29505.05,29578.01,29413.86,29578.01,29575.80187,29742.12439,29948.74941,...,24690.60453,24406.85466,24125.00818,1.32,0.007158,0.036122,-0.121144,0.484661,0.453964,0.438351


In [2]:
maxLevel = 21                           
maxTP = 1                
maxTS = 4096             
nTP = 0                    
p3 = 1.0 / 3.0          

stime = 0
etime = 0
Gstime = 0
Getime = 0
tlapse = 0
Gtlapse = 0

day = 2261

DT_DT= np.zeros((day,1))

DT_OP= np.zeros((day,1))
DT_HI= np.zeros((day,1))
DT_LO= np.zeros((day,1))
DT_CL= np.zeros((day,1))
DT_VL= np.zeros((day,1))

cHurst= np.zeros((day,1))
oHurst= np.zeros((day,1))
hHurst= np.zeros((day,1))
lHurst= np.zeros((day,1))
DT_RT= np.zeros((day,1))

Q= np.zeros((100,1))
NQ= np.zeros((100,1))
r= np.zeros((100,1))


QFEL = np.zeros((21,1))       
QPR = np.zeros((21,1))       
NQPR = np.zeros((21,1))      
K = np.zeros((21,1))


ALL_Pos_QPL = np.zeros((day,21))
ALL_Neg_QPL = np.zeros((day,21))

print("Printout ALL K values K0 .. K20 for first 20 Levels")
for L in range(0,21):    
    K[L] = math.pow((1.1924 + (33.2383 * L) + (56.2169 * L * L)) / (1 + (43.6106 * L)), p3)
    print("Level ",L, " K",L, " = ", K[L])   

Printout ALL K values K0 .. K20 for first 20 Levels
Level  0  K 0  =  [1.06041043]
Level  1  K 1  =  [1.2665998]
Level  2  K 2  =  [1.4911995]
Level  3  K 3  =  [1.66349934]
Level  4  K 4  =  [1.80609901]
Level  5  K 5  =  [1.92919156]
Level  6  K 6  =  [2.03832293]
Level  7  K 7  =  [2.13688129]
Level  8  K 8  =  [2.22710524]
Level  9  K 9  =  [2.31055991]
Level  10  K 10  =  [2.38838747]
Level  11  K 11  =  [2.46144992]
Level  12  K 12  =  [2.53041565]
Level  13  K 13  =  [2.59581465]
Level  14  K 14  =  [2.65807505]
Level  15  K 15  =  [2.71754825]
Level  16  K 16  =  [2.77452666]
Level  17  K 17  =  [2.82925649]
Level  18  K 18  =  [2.88194724]
Level  19  K 19  =  [2.93277882]
Level  20  K 20  =  [2.98190702]


In [3]:
DT_DT = df['Date']     
DT_OP = df['Open']
DT_HI = df['High']
DT_LO = df['Low']
DT_CL = df['Close']
hurst = df['hurst']
hurstp = df['hurstp']
hurstm = df['hurstm']
TSsize = df.shape[0]

TSsize

1711

In [4]:
# For Hurst
maxRno = TSsize - 2

for d in range (0,TSsize - 2):
    if (DT_CL[d + 1] > 0):
        DT_RT[d] = DT_CL[d] / DT_CL[d + 1]          
    else:
        DT_RT[d] = 1
        
mu = 0
for d in range(0,maxRno):
    mu = mu + DT_RT[d]
mu = mu / maxRno

sigma = 0
for d in range(0,maxRno):
    sigma = sigma + (DT_RT[d] - mu) * (DT_RT[d] - mu)
    
sigma = math.sqrt((sigma/ maxRno))

dr = 3 * sigma/ 50

auxR = 0
tQno = 0
num = 0

for nR in range (0,maxRno):    
    bFound = False
    nQ = 0
    # Get the start position of the wave function.
    auxR = 1 - (dr * 50)
    # Get the total number of the returns in each range of each segment of the wave function.
    while((bFound == False) and (nQ < 100)):
        
        if ((DT_RT[nR] > auxR) and (DT_RT[nR] <= (auxR + dr))):
            
            Q[nQ] = Q[nQ] + 1
            tQno= tQno + 1
            bFound = True
        else:
            num = num+1
            nQ = nQ + 1
            auxR = auxR + dr

auxR = 1 - (dr * 50)
for nQ in range(0,100):
    r[nQ] = auxR
    NQ[nQ] = Q[nQ] / tQno        
    auxR = auxR + dr

maxQ = 0
maxQno = 0

for nQ in range(0,100):     
    if (NQ[nQ] > maxQ):
        maxQ = NQ[nQ]
        maxQno = nQ

for i in range(0,day):     
    r0 = r[maxQno] - (dr / 2)
    r1 = r0 + dr
    rn1 = r0 - dr
    
for i in range(0,day):  
    Lup = (pow(rn1, 2) * NQ[maxQno - 1]) - (pow(r1, 2) * NQ[maxQno + 1])
    Ldw = (pow(rn1, 4) * NQ[maxQno - 1]) - (pow(r1, 4) * NQ[maxQno + 1])

L = abs(Lup / Ldw)

for eL in range(0,21):
    p = -1 * pow((2 * eL + 1), 2)
    q = -1 * L * pow((2 * eL + 1), 3) * pow(K[eL], 3)
    u = pow((-0.5 * q + np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    v = pow((-0.5 * q - np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    QFEL[eL] = u + v

for eL in range(0,21):
# Compute the QPR.  
    QPR[eL] = QFEL[eL] / QFEL[0]
#Compute the NQPR.
    NQPR[eL] = 1 + 0.21 * sigma * QPR[eL]

cQHLdata = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "cQHLp1", "cQHLp2", "cQHLp3", "cQHLp4", "cQHLp5", "cQHLp6", "cQHLp7", "cQHLp8",
      "cQHLp9", "cQHLp10", "cQHLp11", "cQHLp12", "cQHLp13", "cQHLp14", "cQHLp15", "cQHLp16", "cQHLp17", "cQHLp18", "cQHLp19", "cQHLp20", "cQHLp21", "cQHLn1", "cQHLn2",
      "cQHLn3", "cQHLn4", "cQHLn5", "cQHLn6", "cQHLn7", "cQHLn8", "cQHLn9", "cQHLn10", "cQHLn11", "cQHLn12", "cQHLn13", "cQHLn14", "cQHLn15", "cQHLn16",
      "cQHLn17", "cQHLn18", "cQHLn19", "cQHLn20", "cQHLn21"])

for d in range(0,TSsize):
    for eL in range(0,21):
        ALL_Pos_QPL[d][eL] = hurst[d] * NQPR[eL]
        ALL_Neg_QPL[d][eL] = hurst[d] / NQPR[eL]

cQHLdata.Date= DT_DT
cQHLdata.Open= DT_OP
cQHLdata.High= DT_HI
cQHLdata.Low = DT_LO
cQHLdata.Close = DT_CL
cqhlv = cQHLdata.values

for d in range(0,TSsize - 2):
    for eL in range(0,21):
        cqhlv[d][eL+5] = ALL_Pos_QPL[d][eL]
        cqhlv[d][eL+26] = ALL_Neg_QPL[d][eL]

cQHLdata = pd.DataFrame(cqhlv,columns=["Date", "Open", "High", "Low", "Close", "cQHLp1", "cQHLp2", "cQHLp3", "cQHLp4", "cQHLp5", "cQHLp6", "cQHLp7", "cQHLp8",
      "cQHLp9", "cQHLp10", "cQHLp11", "cQHLp12", "cQHLp13", "cQHLp14", "cQHLp15", "cQHLp16", "cQHLp17", "cQHLp18", "cQHLp19", "cQHLp20", "cQHLp21", "cQHLn1", "cQHLn2",
      "cQHLn3", "cQHLn4", "cQHLn5", "cQHLn6", "cQHLn7", "cQHLn8", "cQHLn9", "cQHLn10", "cQHLn11", "cQHLn12", "cQHLn13", "cQHLn14", "cQHLn15", "cQHLn16",
      "cQHLn17", "cQHLn18", "cQHLn19", "cQHLn20", "cQHLn21"])

dfc = pd.DataFrame(cQHLdata)
dfc.to_csv("./Datasets/HSI/HSI_qhl.csv",index = True)

In [5]:
# For hurstplus
maxRno = TSsize - 2

for d in range (0,TSsize - 2):
    if (DT_OP[d + 1] > 0):
        DT_RT[d] = DT_OP[d] / DT_OP[d + 1]          
    else:
        DT_RT[d] = 1
        
mu = 0
for d in range(0,maxRno):
    mu = mu + DT_RT[d]
mu = mu / maxRno

sigma = 0
for d in range(0,maxRno):
    sigma = sigma + (DT_RT[d] - mu) * (DT_RT[d] - mu)
    
sigma = math.sqrt((sigma/ maxRno))

dr = 3 * sigma/ 50

auxR = 0
tQno = 0
num = 0

for nR in range (0,maxRno):    
    bFound = False
    nQ = 0
    # Get the start position of the wave function.
    auxR = 1 - (dr * 50)
    # Get the total number of the returns in each range of each segment of the wave function.
    while((bFound == False) and (nQ < 100)):
        
        if ((DT_RT[nR] > auxR) and (DT_RT[nR] <= (auxR + dr))):
            
            Q[nQ] = Q[nQ] + 1
            tQno= tQno + 1
            bFound = True
        else:
            num = num+1
            nQ = nQ + 1
            auxR = auxR + dr

auxR = 1 - (dr * 50)
for nQ in range(0,100):
    r[nQ] = auxR
    NQ[nQ] = Q[nQ] / tQno        
    auxR = auxR + dr

maxQ = 0
maxQno = 0

for nQ in range(0,100):     
    if (NQ[nQ] > maxQ):
        maxQ = NQ[nQ]
        maxQno = nQ

for i in range(0,day):     
    r0 = r[maxQno] - (dr / 2)
    r1 = r0 + dr
    rn1 = r0 - dr
    
for i in range(0,day):  
    Lup = (pow(rn1, 2) * NQ[maxQno - 1]) - (pow(r1, 2) * NQ[maxQno + 1])
    Ldw = (pow(rn1, 4) * NQ[maxQno - 1]) - (pow(r1, 4) * NQ[maxQno + 1])

L = abs(Lup / Ldw)

for eL in range(0,21):
    p = -1 * pow((2 * eL + 1), 2)
    q = -1 * L * pow((2 * eL + 1), 3) * pow(K[eL], 3)
    u = pow((-0.5 * q + np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    v = pow((-0.5 * q - np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    QFEL[eL] = u + v

for eL in range(0,21):
# Compute the QPR.  
    QPR[eL] = QFEL[eL] / QFEL[0]
#Compute the NQPR.
    NQPR[eL] = 1 + 0.21 * sigma * QPR[eL]

cQHLdata = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "oQHLp1", "oQHLp2", "oQHLp3", "oQHLp4", "oQHLp5", "oQHLp6", "oQHLp7", "oQHLp8",
      "oQHLp9", "oQHLp10", "oQHLp11", "oQHLp12", "oQHLp13", "oQHLp14", "oQHLp15", "oQHLp16", "oQHLp17", "oQHLp18", "oQHLp19", "oQHLp20", "oQHLp21", "oQHLn1", "oQHLn2",
      "oQHLn3", "oQHLn4", "oQHLn5", "oQHLn6", "oQHLn7", "oQHLn8", "oQHLn9", "oQHLn10", "oQHLn11", "oQHLn12", "oQHLn13", "oQHLn14", "oQHLn15", "oQHLn16",
      "oQHLn17", "oQHLn18", "oQHLn19", "oQHLn20", "oQHLn21"])

for d in range(0,TSsize):
    for eL in range(0,21):
        ALL_Pos_QPL[d][eL] = hurstp[d] * NQPR[eL]
        ALL_Neg_QPL[d][eL] = hurstp[d] / NQPR[eL]

cQHLdata.Date= DT_DT
cQHLdata.Open= DT_OP
cQHLdata.High= DT_HI
cQHLdata.Low = DT_LO
cQHLdata.Close = DT_CL
cqhlv = cQHLdata.values

for d in range(0,TSsize - 2):
    for eL in range(0,21):
        cqhlv[d][eL+5] = ALL_Pos_QPL[d][eL]
        cqhlv[d][eL+26] = ALL_Neg_QPL[d][eL]

cQHLdata = pd.DataFrame(cqhlv,columns=["Date", "Open", "High", "Low", "Close", "oQHLp1", "oQHLp2", "oQHLp3", "oQHLp4", "oQHLp5", "oQHLp6", "oQHLp7", "oQHLp8",
      "oQHLp9", "oQHLp10", "oQHLp11", "oQHLp12", "oQHLp13", "oQHLp14", "oQHLp15", "oQHLp16", "oQHLp17", "oQHLp18", "oQHLp19", "oQHLp20", "oQHLp21", "oQHLn1", "oQHLn2",
      "oQHLn3", "oQHLn4", "oQHLn5", "oQHLn6", "oQHLn7", "oQHLn8", "oQHLn9", "oQHLn10", "oQHLn11", "oQHLn12", "oQHLn13", "oQHLn14", "oQHLn15", "oQHLn16",
      "oQHLn17", "oQHLn18", "oQHLn19", "oQHLn20", "oQHLn21"])

dfc = pd.DataFrame(cQHLdata)
dfc.to_csv("./Datasets/HSI/HSI_qhlp.csv",index = True)

In [6]:
# For hurstminus
maxRno = TSsize - 2

for d in range (0,TSsize - 2):
    if (DT_HI[d + 1] > 0):
        DT_RT[d] = DT_HI[d] / DT_HI[d + 1]          
    else:
        DT_RT[d] = 1
        
mu = 0
for d in range(0,maxRno):
    mu = mu + DT_RT[d]
mu = mu / maxRno

sigma = 0
for d in range(0,maxRno):
    sigma = sigma + (DT_RT[d] - mu) * (DT_RT[d] - mu)
    
sigma = math.sqrt((sigma/ maxRno))

dr = 3 * sigma/ 50

auxR = 0
tQno = 0
num = 0

for nR in range (0,maxRno):    
    bFound = False
    nQ = 0
    # Get the start position of the wave function.
    auxR = 1 - (dr * 50)
    # Get the total number of the returns in each range of each segment of the wave function.
    while((bFound == False) and (nQ < 100)):
        
        if ((DT_RT[nR] > auxR) and (DT_RT[nR] <= (auxR + dr))):
            
            Q[nQ] = Q[nQ] + 1
            tQno= tQno + 1
            bFound = True
        else:
            num = num+1
            nQ = nQ + 1
            auxR = auxR + dr

auxR = 1 - (dr * 50)
for nQ in range(0,100):
    r[nQ] = auxR
    NQ[nQ] = Q[nQ] / tQno        
    auxR = auxR + dr

maxQ = 0
maxQno = 0

for nQ in range(0,100):     
    if (NQ[nQ] > maxQ):
        maxQ = NQ[nQ]
        maxQno = nQ

for i in range(0,day):     
    r0 = r[maxQno] - (dr / 2)
    r1 = r0 + dr
    rn1 = r0 - dr
    
for i in range(0,day):  
    Lup = (pow(rn1, 2) * NQ[maxQno - 1]) - (pow(r1, 2) * NQ[maxQno + 1])
    Ldw = (pow(rn1, 4) * NQ[maxQno - 1]) - (pow(r1, 4) * NQ[maxQno + 1])

L = abs(Lup / Ldw)

for eL in range(0,21):
    p = -1 * pow((2 * eL + 1), 2)
    q = -1 * L * pow((2 * eL + 1), 3) * pow(K[eL], 3)
    u = pow((-0.5 * q + np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    v = pow((-0.5 * q - np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    QFEL[eL] = u + v

for eL in range(0,21):
# Compute the QPR.  
    QPR[eL] = QFEL[eL] / QFEL[0]
#Compute the NQPR.
    NQPR[eL] = 1 + 0.21 * sigma * QPR[eL]

cQHLdata = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "hQHLp1", "hQHLp2", "hQHLp3", "hQHLp4", "hQHLp5", "hQHLp6", "hQHLp7", "hQHLp8",
      "hQHLp9", "hQHLp10", "hQHLp11", "hQHLp12", "hQHLp13", "hQHLp14", "hQHLp15", "hQHLp16", "hQHLp17", "hQHLp18", "hQHLp19", "hQHLp20", "hQHLp21", "hQHLn1", "hQHLn2",
      "hQHLn3", "hQHLn4", "hQHLn5", "hQHLn6", "hQHLn7", "hQHLn8", "hQHLn9", "hQHLn10", "hQHLn11", "hQHLn12", "hQHLn13", "hQHLn14", "hQHLn15", "hQHLn16",
      "hQHLn17", "hQHLn18", "hQHLn19", "hQHLn20", "hQHLn21"])

for d in range(0,TSsize):
    for eL in range(0,21):
        ALL_Pos_QPL[d][eL] = hurstm[d] * NQPR[eL]
        ALL_Neg_QPL[d][eL] = hurstm[d] / NQPR[eL]

cQHLdata.Date= DT_DT
cQHLdata.Open= DT_OP
cQHLdata.High= DT_HI
cQHLdata.Low = DT_LO
cQHLdata.Close = DT_CL
cqhlv = cQHLdata.values

for d in range(0,TSsize - 2):
    for eL in range(0,21):
        cqhlv[d][eL+5] = ALL_Pos_QPL[d][eL]
        cqhlv[d][eL+26] = ALL_Neg_QPL[d][eL]

cQHLdata = pd.DataFrame(cqhlv,columns=["Date", "Open", "High", "Low", "Close", "hQHLp1", "hQHLp2", "hQHLp3", "hQHLp4", "hQHLp5", "hQHLp6", "hQHLp7", "hQHLp8",
      "hQHLp9", "hQHLp10", "hQHLp11", "hQHLp12", "hQHLp13", "hQHLp14", "hQHLp15", "hQHLp16", "hQHLp17", "hQHLp18", "hQHLp19", "hQHLp20", "hQHLp21", "hQHLn1", "hQHLn2",
      "hQHLn3", "hQHLn4", "hQHLn5", "hQHLn6", "hQHLn7", "hQHLn8", "hQHLn9", "hQHLn10", "hQHLn11", "hQHLn12", "hQHLn13", "hQHLn14", "hQHLn15", "hQHLn16",
      "hQHLn17", "hQHLn18", "hQHLn19", "hQHLn20", "hQHLn21"])

dfc = pd.DataFrame(cQHLdata)
dfc.to_csv("./Datasets/HSI/HSI_qhlm.csv",index = True)